# dependecies

In [1]:
#ml agents

import mlagents_envs
from mlagents_envs.environment import UnityEnvironment
from mlagents_envs.base_env import ActionTuple


#gymnasium
import gymnasium as gym
from gymnasium import spaces

#ray
import ray
from ray import  tune ,train
from ray.rllib.examples.policy.random_policy import RandomPolicy
from ray.rllib.evaluation.rollout_worker import RolloutWorker
from ray.rllib.evaluation.worker_set import WorkerSet
from ray.rllib.env.multi_agent_env import MultiAgentEnv
from ray.rllib.algorithms.algorithm_config import AlgorithmConfig
from ray.rllib.algorithms.algorithm import Algorithm
from ray.tune.registry import register_env
from ray.rllib.policy.policy import PolicySpec
from ray.rllib.env.multi_agent_env import MultiAgentEnv
from ray.rllib.algorithms.ppo.ppo import PPOConfig
from ray.rllib.algorithms.ppo.ppo import PPO
from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.models import MODEL_DEFAULTS
from ray.rllib.policy.rnn_sequencing import add_time_dimension
from ray.rllib.models.torch.misc import SlimFC , SlimConv2d,normc_initializer

from ray.rllib.models.torch.fcnet import  FullyConnectedNetwork
from ray.rllib.models.torch.visionnet import  VisionNetwork
from ray.rllib.models.torch.recurrent_net import LSTMWrapper

from ray.rllib.policy.policy import Policy

# torch
import torch
from torch import nn
import torch.onnx as onnx



#others
import numpy as np
import random
import time
from pprint import pprint
from typing import Any, Dict, List, Optional, Tuple, Union
import os
import time
from ray.rllib.utils.annotations import override




#type = "2_0"
#type = "1.75_0.25"
#type ="1.5_0.5"
#type="1.25_0.75"
#type ="1_1"
#type="0_2"

#type="0"
#type="0.125"
#type="0.25"
#type="0.5"
#type="1"
type="2"











#algo = "2_0"
#algo="1.75_0.25"
#algo ="1.5_0.5"
#algo="1.25_0.75"
#algo ="1_1"
#algo="0_2"

#algo="00"
#algo="0.125"
#algo="0.25"
#algo="0.5"
#algo="1"
algo="2"

episode_horizon=1000000
mapsize=8
env_out=139
infosize=3
observationsize=env_out-infosize
disc_0=7
#disc_1=3



filename_window="C:\\my\\code\\github reposetory\\Continios_Level_Design\\unity_built_window\\RLUnity"
filename_server="C:\\my\\code\\github reposetory\\Continios_Level_Design\\unity_built_window_server\\"+type+"\\RLUnity"
file=filename_server
filename_algo ="C:\\my\\code\\github reposetory\\Continios_Level_Design\\Algo\\"+algo
algoname=filename_algo












# env

In [ ]:
class Unity3DEnv(MultiAgentEnv):

    
    _BASE_PORT_EDITOR = 5004
    _BASE_PORT_ENVIRONMENT = 5005
    _WORKER_ID = 0

    def __init__(
        self,
        file_name: str = None,
        port: Optional[int] = None,
        seed: int = 0,
        no_graphics: bool = False,
        timeout_wait: int = 300,
        episode_horizon: int = None,

    ):

        self._skip_env_checking = True
        super().__init__()

        if file_name is None:
            print(
                "No game binary provided, will use a running Unity editor "
                "instead.\nMake sure you are pressing the Play (|>) button in "
                "your editor to start."
            )

        # Try connecting to the Unity3D game instance. If a port is blocked
        port_ = None
        while True:
            # Sleep for random time to allow for concurrent startup of many
            # environments (num_workers >> 1). Otherwise, would lead to port
            # conflicts sometimes.
            if port_ is not None:
                time.sleep(random.randint(1, 10))
            port_ = port or (
                self._BASE_PORT_ENVIRONMENT if file_name else self._BASE_PORT_EDITOR
            )
            # cache the worker_id and
            # increase it for the next environment
            worker_id_ = Unity3DEnv._WORKER_ID if file_name else 0
            Unity3DEnv._WORKER_ID += 1
            try:
                self.unity_env = UnityEnvironment(
                    file_name=file_name,
                    worker_id=worker_id_,
                    base_port=port_,
                    seed=seed,
                    no_graphics=no_graphics,
                    timeout_wait=timeout_wait,
                )

                print("Created UnityEnvironment for port {}".format(port_ + worker_id_))
            except mlagents_envs.exception.UnityWorkerInUseException:
                pass
            else:
                break

        self.episode_horizon = episode_horizon
        self.episode_timesteps = 0






    def step(
        self, action_dict
    ):

        # it take a action_dict {"instantiator Behaviour" :  dict{"element_1":spaces.box , "element_2":spaces.discrete }  , 
        #                                   modifyer Behavior":         dict{"element_1":spaces.box , "element_2":spaces.discrete } }
        
        # make a list called actions for each beahivor for example
        # for "instantiator Behaviour" make : action[0] = dict{"element_1":spaces.box , "element_2":spaces.discrete }
        # for "modifyer Behavior" make : action[0] =  dict{"element_1":spaces.box , "element_2":spaces.discrete } }

        for behavior_name in self.unity_env.behavior_specs:
                actions = []
                for agent_id in self.unity_env.get_steps(behavior_name)[0].agent_id:
                    key = behavior_name + "_{}".format(agent_id)
                    actions.append(action_dict[key])



                    #the behaviorname == actuall behaviorname in unity +team=0
                    # for action tuple we had to pass a numpy arrye with shape (<number of agents>, <action size>) ___> for box in []  for discrete in [[]]
                    if behavior_name=="instantiator Behaviour?team=0":
                       x=actions[0]["element_1"]
                       #y=actions[0]["element_2"]
                       discrete=np.array([[x]])
                        
                       action_tuple =ActionTuple( discrete= discrete )

                   



                    # pass the current behavior and its action_tuple to step() function
                    self.unity_env.set_actions(behavior_name, action_tuple)

        self.unity_env.step()
        self.episode_timesteps += 1
        obs, rewards, terminateds, truncateds, infos = self._get_step_results()
        return obs, rewards, terminateds, truncateds, infos














    def reset(
        self, *, seed=None, options=None
    ):
        self.episode_timesteps = 0
        self.unity_env.reset()
        obs, _, _, _, infos = self._get_step_results()
        return obs, infos








    def _get_step_results(self):

        # first we set the initial dict that step return
        obs = {}
        rewards = {}
        terminated ={}
        truncated={}
        infos = {}


        num_active=0
        num_done=0
        num_all=0



        #go thorugh all the behavior
        # return decision_step (batch of agent have similar behavior)
        # return trminal steps  (batch of agent have similar behavior /that end thier episode)
        for behavior_name  in self.unity_env.behavior_specs:
            decision_steps, terminal_steps = self.unity_env.get_steps(behavior_name)
            num_active=num_active+len(decision_steps)
            num_done=num_done+len(terminal_steps)
            num_all=num_active+num_done





            #set the obs / reward  from  decision step for each agent like:{"behavior_name+agent_id" : numpy array}
            for agent_id, idx in decision_steps.agent_id_to_index.items():
                key = behavior_name + "_{}".format(agent_id)
                os = tuple(o[idx] for o in decision_steps.obs)
                os = os[0] if len(os) == 1 else os
                obs[key]=  os[infosize:]
                #infos={ "satisfied": os[0], "w_l": os[1]  , "max_percentage":os[2]}


                
                rewards[key] = (
                    decision_steps.reward[idx] + decision_steps.group_reward[idx]
                )



            #set the obs / reward  from  terminal_steps for each agent like:{"behavior_name+agent_id" : numpy array}
            for agent_id, idx in terminal_steps.agent_id_to_index.items():
                key = behavior_name + "_{}".format(agent_id)
                if key not in obs:
                    os = tuple(o[idx] for o in terminal_steps.obs)
                    os = os[0] if len(os) == 1 else os
                    obs[key]=  os[infosize:]
                    #infos={ "satisfied": os[0], "w_l": os[1]  , "max_percentage":os[2]}
                    
                rewards[key] = (
                    terminal_steps.reward[idx] + terminal_steps.group_reward[idx]
                )

        
            #+ decision_steps.group_reward[idx]
         #+ terminal_steps.group_reward[idx]


     



        # set the terminated if all agent end thier own episod like: {"__all__":True}
        if(num_active>0):
           terminated["__all__"]=False
        else:
           terminated["__all__"]=True



        # set the truncated /if the step of the env reach the horizen like{"__all__":True}
        if self.episode_timesteps == self.episode_horizon:
          truncated["__all__"]=True
        else:
          truncated["__all__"]=False


        # return
        return obs, rewards, terminated, truncated, infos


# env_info

In [2]:
class Unity3DEnv(MultiAgentEnv):

    
    _BASE_PORT_EDITOR = 5004
    _BASE_PORT_ENVIRONMENT = 5005
    _WORKER_ID = 0

    def __init__(
        self,
        file_name: str = None,
        port: Optional[int] = None,
        seed: int = 0,
        no_graphics: bool = False,
        timeout_wait: int = 300,
        episode_horizon: int = None,

    ):

        self._skip_env_checking = True
        super().__init__()

        if file_name is None:
            print(
                "No game binary provided, will use a running Unity editor "
                "instead.\nMake sure you are pressing the Play (|>) button in "
                "your editor to start."
            )

        # Try connecting to the Unity3D game instance. If a port is blocked
        port_ = None
        while True:
            # Sleep for random time to allow for concurrent startup of many
            # environments (num_workers >> 1). Otherwise, would lead to port
            # conflicts sometimes.
            if port_ is not None:
                time.sleep(random.randint(1, 10))
            port_ = port or (
                self._BASE_PORT_ENVIRONMENT if file_name else self._BASE_PORT_EDITOR
            )
            # cache the worker_id and
            # increase it for the next environment
            worker_id_ = Unity3DEnv._WORKER_ID if file_name else 0
            Unity3DEnv._WORKER_ID += 1
            try:
                self.unity_env = UnityEnvironment(
                    file_name=file_name,
                    worker_id=worker_id_,
                    base_port=port_,
                    seed=seed,
                    no_graphics=no_graphics,
                    timeout_wait=timeout_wait,
                )

                print("Created UnityEnvironment for port {}".format(port_ + worker_id_))
            except mlagents_envs.exception.UnityWorkerInUseException:
                pass
            else:
                break

        self.episode_horizon = episode_horizon
        self.episode_timesteps = 0






    def step(
        self, action_dict
    ):

        # it take a action_dict {"instantiator Behaviour" :  dict{"element_1":spaces.box , "element_2":spaces.discrete }  , 
        #                                   modifyer Behavior":         dict{"element_1":spaces.box , "element_2":spaces.discrete } }
        
        # make a list called actions for each beahivor for example
        # for "instantiator Behaviour" make : action[0] = dict{"element_1":spaces.box , "element_2":spaces.discrete }
        # for "modifyer Behavior" make : action[0] =  dict{"element_1":spaces.box , "element_2":spaces.discrete } }

        for behavior_name in self.unity_env.behavior_specs:
                actions = []
                for agent_id in self.unity_env.get_steps(behavior_name)[0].agent_id:
                    key = behavior_name + "_{}".format(agent_id)
                    actions.append(action_dict[key])



                    #the behaviorname == actuall behaviorname in unity +team=0
                    # for action tuple we had to pass a numpy arrye with shape (<number of agents>, <action size>) ___> for box in []  for discrete in [[]]
                    if behavior_name=="instantiator Behaviour?team=0":
                       x=actions[0]["element_1"]
                       #y=actions[0]["element_2"]
                       discrete=np.array([[x]])
                        
                       action_tuple =ActionTuple( discrete= discrete )

                   



                    # pass the current behavior and its action_tuple to step() function
                    self.unity_env.set_actions(behavior_name, action_tuple)

        self.unity_env.step()
        self.episode_timesteps += 1
        obs, rewards, terminateds, truncateds, infos = self._get_step_results()
        return obs, rewards, terminateds, truncateds, infos














    def reset(
        self, *, seed=None, options=None
    ):
        self.episode_timesteps = 0
        self.unity_env.reset()
        obs, _, _, _, infos = self._get_step_results()
        return obs, infos








    def _get_step_results(self):

        # first we set the initial dict that step return
        obs = {}
        rewards = {}
        terminated ={}
        truncated={}
        infos = {}


        num_active=0
        num_done=0
        num_all=0



        #go thorugh all the behavior
        # return decision_step (batch of agent have similar behavior)
        # return trminal steps  (batch of agent have similar behavior /that end thier episode)
        for behavior_name  in self.unity_env.behavior_specs:
            decision_steps, terminal_steps = self.unity_env.get_steps(behavior_name)
            num_active=num_active+len(decision_steps)
            num_done=num_done+len(terminal_steps)
            num_all=num_active+num_done





            #set the obs / reward  from  decision step for each agent like:{"behavior_name+agent_id" : numpy array}
            for agent_id, idx in decision_steps.agent_id_to_index.items():
                key = behavior_name + "_{}".format(agent_id)
                os = tuple(o[idx] for o in decision_steps.obs)
                os = os[0] if len(os) == 1 else os
                obs[key]=  os[infosize:]
                infos={ "satisfied": os[0], "w_l": os[1]  , "max_percentage":os[2]}


                
                rewards[key] = (
                    decision_steps.reward[idx] + decision_steps.group_reward[idx]
                )



            #set the obs / reward  from  terminal_steps for each agent like:{"behavior_name+agent_id" : numpy array}
            for agent_id, idx in terminal_steps.agent_id_to_index.items():
                key = behavior_name + "_{}".format(agent_id)
                if key not in obs:
                    os = tuple(o[idx] for o in terminal_steps.obs)
                    os = os[0] if len(os) == 1 else os
                    obs[key]=  os[infosize:]
                    infos={ "satisfied": os[0], "w_l": os[1]  , "max_percentage":os[2]}
                    
                rewards[key] = (
                    terminal_steps.reward[idx] + terminal_steps.group_reward[idx]
                )

        
            #+ decision_steps.group_reward[idx]
         #+ terminal_steps.group_reward[idx]


     



        # set the terminated if all agent end thier own episod like: {"__all__":True}
        if(num_active>0):
           terminated["__all__"]=False
        else:
           terminated["__all__"]=True



        # set the truncated /if the step of the env reach the horizen like{"__all__":True}
        if self.episode_timesteps == self.episode_horizon:
          truncated["__all__"]=True
        else:
          truncated["__all__"]=False


        # return
        return obs, rewards, terminated, truncated, infos


# test 

In [ ]:
env =Unity3DEnv(file_name=filename_window , 
                              episode_horizon=episode_horizon  , 
                                no_graphics=False,
               )

## test reset()

In [ ]:

obs,infos=env.reset()

#print(obs[0]['instantiator Behaviour?team=0_0'][:])
#print("\n")

print(obs['instantiator Behaviour?team=0_0'])
print("\n")
print("\n")
print("\n")
print(infos)

## test step()

In [ ]:

sample=spaces.Discrete(disc_0).sample()
print(sample)
actions_dict={'instantiator Behaviour?team=0_0':{"element_1":sample, 
                                                                            
                                                                              } 
                        }

obs,rew,terminated,truncated,info=env.step(actions_dict)



print(obs['instantiator Behaviour?team=0_0'][:])
print("\n")




# rollout with random policy

In [ ]:

policies = { "policy_initializer": (RandomPolicy,
                                            spaces.Box(float("-inf"), float("inf"), (observationsize,)),
                                                
                                    spaces.Dict({
                                                 "element_1":spaces.Discrete(disc_0),
                                                  
                                                }),
                                    AlgorithmConfig())

                }





def policy_mapping_fn(agent_id, episode, worker, **kwargs):
    if agent_id == 'instantiator Behaviour?team=0_0':
        return "policy_initializer"
    if agent_id =='modifyer Behavior?team=0_1':
        return "policy_modifier"


config=AlgorithmConfig().multi_agent(policies=policies,policy_mapping_fn=policy_mapping_fn) .rollouts (rollout_fragment_length=70)

#batch_mode="complete_episodes"


worker=RolloutWorker(env_creator=lambda _: Unity3DEnv(file_name=filename_window  ,
                                                      episode_horizon =episode_horizon ,
                                                      no_graphics=False,
                                                     ),
                   
config=config)





In [ ]:
multi_batchs=[]
for i in range(1):
    multi_batch=worker.sample()  
    multi_batchs.append(multi_batch)
    #print("obs is {}".format(multi_batch["policy_initializer"]["obs"]))
    #print("actions is{}".format(multi_batch["policy_initializer"]["actions"]))
    #print("terminateds is{}".format(multi_batch["policy_initializer"]["terminateds"]))
    #print("truncateds is{}".format(multi_batch["policy_initializer"]["truncateds"]))
    print("rewards is{}".format(multi_batch["policy_initializer"]["rewards"]))
    total = sum(n for n in multi_batch["policy_initializer"]["rewards"])
    print(total)
    
    

   
   
  

# register env

In [3]:
# register the unity env
tune.register_env("unity3d",lambda _: Unity3DEnv(file_name=file,
                                                                                 episode_horizon=episode_horizon,
                                                                                    no_graphics=False,
                                                                                )
                                                )

# model

# FC_lstm

In [ ]:
class mine(  TorchModelV2, nn.Module):
    
    def __init__(self, obs_space, action_space, num_outputs, model_config, name):
        TorchModelV2.__init__( self, obs_space, action_space, num_outputs, model_config, name)
        nn.Module.__init__(self)
    
        self._features=None
    
  

        self.fc_grid_1=SlimConv2d(
                        in_channels=1,
                        out_channels=16,
                        kernel=3,
                        stride=1,
                        padding=1,
                        activation_fn="relu",
                    )

        self.fc_grid_2=SlimConv2d(
                        in_channels=16,
                        out_channels=32,
                        kernel=3,
                        stride=1,
                        padding=1,
                        activation_fn="relu",
                    )
       
        
     
        self.fc=SlimFC(
            in_size=3200   ,
            out_size=256,
            initializer=normc_initializer(0.01), 
            activation_fn="relu",
        )
  

        #fc layer
        self.fc1=SlimFC(
            in_size=(observationsize)-(mapsize*mapsize),
            out_size=64,
            initializer=normc_initializer(0.01), 
            activation_fn="relu",
        )

      


        self.concated=SlimFC(
            in_size=256+64,
            out_size=256,
            initializer=normc_initializer(0.01), 
            activation_fn="relu",
        )
        

      
      
 

        #logit
        self.logit =   SlimFC(
            in_size= 256,
            out_size=num_outputs,
            initializer=normc_initializer(0.01), 
            activation_fn=None,
        )
      

        
        # value
        self.value=SlimFC(
            in_size=256,
            out_size=1,
            activation_fn=None,
           initializer=normc_initializer(0.01), 
        )


    
    def forward(self, input_dict, state, seq_lens):

        self.fc_input_grid=  input_dict["obs_flat"][:,:mapsize*mapsize].float()
        self.fc_input_grid= self.fc_input_grid.reshape((self.fc_input_grid.shape[0], 1, mapsize ,mapsize))
    
      
        
        self.fc_input=  input_dict["obs_flat"][:,mapsize*mapsize:].float()
      
        #fc_grid
        out_grid=self.fc_grid_1(self.fc_input_grid)
        out_grid=self.fc_grid_2(out_grid)
        out_grid=out_grid.reshape(out_grid.shape[0],out_grid.shape[1]*out_grid.shape[2]*out_grid.shape[3])
        out_grid=self.fc(out_grid)
        
      
        #fc 
        fc_out=self.fc1(self.fc_input)

       
        





        #concat them  and pass to self.cocncated
        out= torch.cat((  fc_out, out_grid), dim=1)
        self._features=self.concated(out)
     


        
        # pass the feature to the logit  return out
        output = self.logit(self._features)
     

        return output , state




    
   


    
    def value_function(self) :
        assert  self._features is not None, "must call forward() first"
        return torch.reshape(self.value(self._features), [-1])


# test

In [ ]:
model = mine(obs_space=  spaces.Box(float("-inf"), float("inf"), (observationsize,)),
                                            action_space=  spaces.Dict({
                                              "element_1":spaces.MultiDiscrete([disc_0,4]),
                                                }),
                                            num_outputs=9, 
                                            model_config=MODEL_DEFAULTS,
                                            name="my_model")


# initialize  input_dict  , seq_lent
obs = torch.rand(6,observationsize) 
#print(obs)
input_dict={"obs_flat":obs , "obs":obs }
seq_lens=np.array([0,])

#initialize state with  get_initial_state
state = model.get_initial_state()

# initialize sate with fake state
h0=torch.rand(1,128)
c0=torch.rand(1,128)
state=[h0,c0]


# foward 
out = model(
            input_dict=input_dict, 
            state=state, 
            seq_lens=seq_lens
           )

#print("the output is {}".format(out))

# register the model

In [ ]:
## register model|"

ModelCatalog.register_custom_model("mine", mine)


# export algo and policy

In [4]:
def export_algo():
  # save and print the path of the checkpoint
  save_result = algo.save()
  path_to_checkpoint = save_result.checkpoint.path
  print(
      "An Algorithm checkpoint has been created inside directory: "
      f"'{path_to_checkpoint}'."
  )
  start_time = time.time()
  while not os.path.exists(path_to_checkpoint):
      if time.time() - start_time >= 20:
          print("time for build algo out")
          break
      time.sleep(1)
  print("algo  builded at {}".format(path_to_checkpoint) )
  return path_to_checkpoint



def download_algo(path_to_algo):
  !zip -r {path_to_algo}.zip {path_to_algo}
  start_time = time.time()
  while not os.path.exists(path_to_algo):
      if time.time() - start_time >= 20:
          print("time for zipping algo out")
          break
      time.sleep(1)
  print("algo  zipped at {}".format(path_to_algo) )
  path_to_algo=path_to_algo+".zip"
  files.download(path_to_algo)



def export_policy():
    policy = algo.get_policy(policy_id="policy_initializer")
    policy.export_checkpoint(filename_policy)



#export_policy()

# train

In [ ]:
ray.init()
resources = ray.cluster_resources()
print(resources)

import torch
print(torch.cuda.is_available()) 


## tune

In [ ]:

policies = { "policy_initializer":PolicySpec (
                                    observation_space=spaces.Box(float("-inf"), float("inf"), (observationsize,)),
                                                
                                    action_space=spaces.Dict({
                                                 "element_1":spaces.Discrete(5),
                                                 "element_2":  spaces.Box(low=-2, high=2,shape= (2,)),
                                                }),
                                    config={}
                                                                )
                                                         


                }



def policy_mapping_fn(agent_id, episode, worker, **kwargs):
    if agent_id == 'instantiator Behaviour?team=0_0':
        return "policy_initializer"
    if agent_id =='modifyer Behavior?team=0_1':
        return "policy_modifier"







config=(
    PPOConfig()
    .environment("unity3d")
    
    .multi_agent(
                        policies=policies,
                         policy_mapping_fn=policy_mapping_fn
                        )
    
    .framework("torch")
    
    .rollouts(
            num_rollout_workers=15,
            batch_mode="complete_episodes",
            rollout_fragment_length='auto' ,
                )
    
    .resources(num_gpus=1)
    
    .training(
            lr=0.0003,
            lambda_=0.95,
            gamma=0.99,
            sgd_minibatch_size=4096,
            train_batch_size=32768,
            num_sgd_iter=30,
            clip_param=0.3,
            model={
                    #"custom_model": "mine", 
                    #"custom_model_config": {},
                    #"conv_filters": [[16, [2,2 ], 1], [64, [4, 4], 3], [128, [1, 1], 5]],
                    "fcnet_hiddens": [512],
                   "use_lstm": True,
                   "lstm_cell_size": 512,
                  "max_seq_len":tune.grid_search([30,200]),
                   #"lstm_use_prev_action": False,
                   # "lstm_use_prev_reward": False,
                   #"_disable_action_flattening":False,

                   },
          

            )
    .evaluation()
    .experimental(_enable_new_api_stack=False)
    .checkpointing(export_native_model_files=True)

    )







In [ ]:
result_grid = tune.Tuner(

    "PPO",
    
     tune_config=tune.TuneConfig(
        num_samples=1,
    ),
      
    param_space=config.to_dict(),

    run_config=train.RunConfig(
        stop={"episode_reward_mean": 20,"training_iteration": 20 } 
                              ),
    
).fit()



In [ ]:

# Iterate over results
for i, result in enumerate(result_grid):
    print(
        f"Trial {i} finished mean reward:",
        result.metrics["episode_reward_mean"]
    )




best_result = result_grid.get_best_result(metric="episode_reward_mean", mode="min")
#print(best_result)
best_reward = best_result.metrics["episode_reward_mean"]
print(best_reward)

## train 

In [ ]:
policies = { "policy_initializer":PolicySpec (
                                    observation_space=spaces.Box(float("-inf"), float("inf"), (observationsize,)),
                                                
                                    action_space=spaces.Dict({
                                                 "element_1":spaces.Discrete(disc_0)
                                        #spaces.MultiDiscrete([disc_0,disc_1]),
                                         #spaces.Discrete(disc_0)
                                           
                                                }),
                                    config={}
                                                                )
                                                         


                }





def policy_mapping_fn(agent_id, episode, worker, **kwargs):
    if agent_id == 'instantiator Behaviour?team=0_0':
        return "policy_initializer"
    if agent_id =='modifyer Behavior?team=0_1':
        return "policy_modifier"




config=(
PPOConfig()
    .environment("unity3d")
    
    .multi_agent(
                        policies=policies,
                         policy_mapping_fn=policy_mapping_fn
                        )
    
    .framework("torch")
    
    .rollouts(
            num_rollout_workers=16,
            batch_mode="complete_episodes",
            #rollout_fragment_length='auto',
        
                )
    
    .resources(num_gpus=1 ,
                      num_gpus_per_worker=0)
    
    .training(
            lr=0.00005,
            lambda_=0.95,
            gamma=1,
            sgd_minibatch_size=1024,
            train_batch_size=1024*16,
            num_sgd_iter=30,   
            clip_param=0.3,
            entropy_coeff=0.,
            model={
                   #"custom_model": "mine",
                   # "custom_model_config": {},
                   #"conv_filters": [[32, [2,2], 1], [64, [2, 2], 1], [128, [1, 1], 1]],
                   "fcnet_hiddens": [256,512,256],
                    #"use_lstm": True,
                    #"lstm_cell_size": 256,
                    #"max_seq_len":500,
                    
                   #"lstm_use_prev_action": True,
                   #"lstm_use_prev_reward": True,
                  # "_disable_action_flattening":True,

                   },
          

            )
    .evaluation()
    .experimental(_enable_new_api_stack=False)
    .checkpointing(export_native_model_files=True)

    )

for key , item in config.items():
    print(key +":{}".format( item) )

In [ ]:
algo=config.build()

In [ ]:
results=[]
for i in range(302):
            result = algo.train()
            results.append(result)
            #pprint(results[i])
            returns=results[i]['sampler_results']['hist_stats']['episode_reward']
    
            
            print("traning iteration number {} finished".format(i))
            print("the min reward for result{} is: {}".format(i,results[i]['episode_reward_min']))
            print("the mean reward for result{} is: {}".format(i,results[i]['episode_reward_mean']))
            print("the max reward for result{} is: {} ".format(i,results[i]['episode_reward_max']))
    
        
            print("\n\n\n\n\n\n")
    
            if (i % 20 == 0):
                export_algo()


        


            if (
                result['timesteps_total'] >= 10000000000
                or result["episode_reward_mean"] >= 10000000000
                or result["episodes_total"]>=100000000000
            ):
                break

In [ ]:
x=results[0]['sampler_results'][ 'hist_stats'] [ 'episode_lengths']

#for number in x:
#    if number != 64:
#        print(number)

# test the algo - policy - model  with fake  observation and state

## algo

In [ ]:
# get the model and policy 
policy=algo.get_policy("policy_initializer")
model=policy.model


# initial state
state=model.get_initial_state()


# initial observation
obs = torch.rand(56) 
obs={"instantiator Behaviour?team=0_0":obs}



# calclute the actions
actions=algo.compute_actions(observations=obs,
                    state=state,
                    policy_id= "policy_initializer")


#print the h0 
pprint(actions[1]['instantiator Behaviour?team=0_0'][0].shape)
s
#print the action

## policy

In [ ]:

policy=algo.get_policy("policy_initializer")

state=model.get_initial_state()
obs = torch.rand(6,56) 
input_dict={"obs_flat":obs , "obs":obs , "state_in":state }
seq=torch.rand(1,30) 


## with input dict
actions,state_out=policy.compute_actions_from_input_dict(input_dict=input_dict)

#with single action
policy.compute_single_action()

#with actions
policy.compute_actions()

## model

In [ ]:

# take the model and policy
policy=algo.get_policy("policy_initializer")
model=policy.model
#pprint(model.view_requirements)

print(model)



# initial state with  get_initial_state
state=model.get_initial_state()
#print(state[0].shape)

# fake initial satate
h_0=torch.rand(1,256) 
c_0=torch.rand(1,256) 
state=(h_0,c_0)


#initial inputdict
obs = torch.rand(5,56) 
input_dict={"obs_flat":obs , "obs":obs }


#initial seq lenth
seq=torch.rand(1,30) 





# foward
out,state=model.forward(input_dict=input_dict, 
                       state=state, 
                       seq_lens=seq)

print(out.shape)




# train loaded algorithm

In [ ]:
def load_algo(path_to_checkpoint):
    algo = Algorithm.from_checkpoint(path_to_checkpoint)
    return algo

algo=load_algo(algoname)

In [ ]:
###### %%capture
results=[]
for i in range(142):
            result = algo.train()
            results.append(result)
            #pprint(results[i])
            print("traning iteration number {} finished".format(i))
            print("the min reward for result{} is: {}".format(i,results[i]['episode_reward_min']))
            print("the mean reward for result{} is: {}".format(i,results[i]['episode_reward_mean']))
            print("the max reward for result{} is: {} ".format(i,results[i]['episode_reward_max']))
    
        
            print("\n\n\n")


            if (i % 20 == 0):
                export_algo()
                
                


        


            if (
                result['timesteps_total'] >= 10000000000
                or result["episode_reward_mean"] >= 10000000000
                or result["episodes_total"]>=100000000000
            ):
                break

# load and test trained algorithm 

In [5]:
env =Unity3DEnv(file_name=filename_window , 
                              episode_horizon=episode_horizon  , 
                                no_graphics=False,
               )

Created UnityEnvironment for port 5021


## test the algorithm

In [6]:

def load_algo(path_to_checkpoint):
   algo = Algorithm.from_checkpoint(path_to_checkpoint)
   return algo

algo=load_algo(filename_algo)


C:\Users\dania\anaconda3\envs\mine\lib\site-packages\ray\rllib\algorithms\algorithm.py:483: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
C:\Users\dania\anaconda3\envs\mine\lib\site-packages\ray\tune\logger\unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
C:\Users\dania\anaconda3\envs\mine\lib\site-packages\ray\tune\logger\unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed i

(RolloutWorker pid=36316) [UnityMemory] Configuration Parameters - Can be set up in boot.config
(RolloutWorker pid=36316)     "memorysetup-bucket-allocator-granularity=16"
(RolloutWorker pid=36316)     "memorysetup-bucket-allocator-bucket-count=8"
(RolloutWorker pid=36316)     "memorysetup-bucket-allocator-block-size=4194304"
(RolloutWorker pid=36316)     "memorysetup-bucket-allocator-block-count=1"
(RolloutWorker pid=36316)     "memorysetup-main-allocator-block-size=16777216"
(RolloutWorker pid=36316)     "memorysetup-thread-allocator-block-size=16777216"
(RolloutWorker pid=36316)     "memorysetup-gfx-main-allocator-block-size=16777216"
(RolloutWorker pid=36316)     "memorysetup-gfx-thread-allocator-block-size=16777216"
(RolloutWorker pid=36316)     "memorysetup-cache-allocator-block-size=4194304"
(RolloutWorker pid=36316)     "memorysetup-typetree-allocator-block-size=2097152"
(RolloutWorker pid=36316)     "memorysetup-profiler-bucket-allocator-granularity=16"
(RolloutWorker pid=3631

(RolloutWorker pid=15592) E0823 19:03:39.350000000 20004 src/core/ext/transport/chttp2/server/chttp2_server.cc:1080] UNKNOWN:No address added out of total 1 resolved for '[::]:5019' {created_time:"2024-08-23T15:33:39.350413683+00:00", children:[UNKNOWN:Failed to add port to server {created_time:"2024-08-23T15:33:39.350406624+00:00", children:[UNAVAILABLE:WSA Error {syscall:"bind", os_error:"Only one usage of each socket address (protocol/network address/port) is normally permitted.\r\n", grpc_status:14, wsa_error:10048, created_time:"2024-08-23T15:33:39.35036507+00:00"}]}]} [repeated 9x across cluster]
(RolloutWorker pid=36132) E0823 19:03:39.382000000 36136 src/core/ext/transport/chttp2/server/chttp2_server.cc:1080] UNKNOWN:No address added out of total 1 resolved for '[::]:5023' {created_time:"2024-08-23T15:33:39.382625057+00:00", children:[UNKNOWN:Failed to add port to server {created_time:"2024-08-23T15:33:39.382618189+00:00", children:[UNAVAILABLE:WSA Error {created_time:"2024-08-

(RolloutWorker pid=36316) - Loaded All Assemblies, in  4.407 seconds
(RolloutWorker pid=36316) - Finished resetting the current domain, in  0.002 seconds
(RolloutWorker pid=36212) [UnityMemory] Configuration Parameters - Can be set up in boot.config
(RolloutWorker pid=36212)  [repeated 30x across cluster]
(RolloutWorker pid=36212) Mono path[0] = 'C:/my/code/github reposetory/Continios_Level_Design/unity_built_window_server/2/RLUnity_Data/Managed'
(RolloutWorker pid=36212) Mono config path = 'C:/my/code/github reposetory/Continios_Level_Design/unity_built_window_server/2/MonoBleedingEdge/etc'
(RolloutWorker pid=36316) ERROR: Shader Sprites/Default shader is not supported on this GPU (none of subshaders/fallbacks are suitable)
(RolloutWorker pid=36316) Microsoft Media Foundation video decoding to texture disabled: graphics device is Null, only Direct3D 11 and Direct3D 12 (only on desktop) are supported for hardware-accelerated video decoding.
(RolloutWorker pid=36316) ERROR: Shader Sprit

(RolloutWorker pid=36256) E0823 19:03:44.415000000 36104 src/core/ext/transport/chttp2/server/chttp2_server.cc:1080] UNKNOWN:No address added out of total 1 resolved for '[::]:5022' {created_time:"2024-08-23T15:33:44.415647034+00:00", children:[UNKNOWN:Failed to add port to server {created_time:"2024-08-23T15:33:44.41563983+00:00", children:[UNAVAILABLE:WSA Error {syscall:"bind", os_error:"Only one usage of each socket address (protocol/network address/port) is normally permitted.\r\n", grpc_status:14, wsa_error:10048, created_time:"2024-08-23T15:33:44.415590837+00:00"}]}]} [repeated 9x across cluster]
(RolloutWorker pid=36172) E0823 19:03:45.020000000 36176 src/core/ext/transport/chttp2/server/chttp2_server.cc:1080] UNKNOWN:No address added out of total 1 resolved for '[::]:5019' {created_time:"2024-08-23T15:33:45.019982439+00:00", children:[UNKNOWN:Failed to add port to server {created_time:"2024-08-23T15:33:45.019975961+00:00", children:[UNAVAILABLE:WSA Error {created_time:"2024-08-

...

(RolloutWorker pid=37568) E0823 19:05:21.291000000 38004 src/core/ext/transport/chttp2/server/chttp2_server.cc:1080] UNKNOWN:No address added out of total 1 resolved for '[::]:5035' {created_time:"2024-08-23T15:35:21.29146063+00:00", children:[UNKNOWN:Failed to add port to server {created_time:"2024-08-23T15:35:21.291452429+00:00", children:[UNAVAILABLE:WSA Error {syscall:"bind", os_error:"Only one usage of each socket address (protocol/network address/port) is normally permitted.\r\n", grpc_status:14, wsa_error:10048, created_time:"2024-08-23T15:35:21.291405936+00:00"}]}]} [repeated 3x across cluster]
(RolloutWorker pid=36308) E0823 19:05:24.334000000 36304 src/core/ext/transport/chttp2/server/chttp2_server.cc:1080] UNKNOWN:No address added out of total 1 resolved for '[::]:5035' {created_time:"2024-08-23T15:35:24.334528096+00:00", children:[UNKNOWN:Failed to add port to server {created_time:"2024-08-23T15:35:24.334520188+00:00", children:[UNAVAILABLE:WSA Error {created_time:"2024-08-

(RolloutWorker pid=37568) [UnityMemory] Configuration Parameters - Can be set up in boot.config
(RolloutWorker pid=37568) Mono path[0] = 'C:/my/code/github reposetory/Continios_Level_Design/unity_built_window_server/2/RLUnity_Data/Managed'
(RolloutWorker pid=37568) Mono config path = 'C:/my/code/github reposetory/Continios_Level_Design/unity_built_window_server/2/MonoBleedingEdge/etc'
(RolloutWorker pid=37568)     "memorysetup-temp-allocator-size-gfx=262144" [repeated 31x across cluster]
(RolloutWorker pid=37568) Found 1 interfaces on host : 0) 192.168.1.33
(RolloutWorker pid=37568) Player connection [17568] Multi-casting "[IP] 192.168.1.33 [Port] 55016 [Flags] 2 [Guid] 3319747385 [EditorId] 1352096587 [Version] 1048832 [Id] WindowsServer(44,dania) [Debug] 0 [PackageName] WindowsServer [ProjectName] RLUnity" to [225.0.0.222:54997]...
(RolloutWorker pid=37568) [Physics::Module] Initialized MultithreadedJobDispatcher with 15 workers.
(RolloutWorker pid=37568) Initialize engine version: 2

(RolloutWorker pid=36308) E0823 19:05:31.347000000 36304 src/core/ext/transport/chttp2/server/chttp2_server.cc:1080] UNKNOWN:No address added out of total 1 resolved for '[::]:5036' {created_time:"2024-08-23T15:35:31.347742363+00:00", children:[UNKNOWN:Failed to add port to server {created_time:"2024-08-23T15:35:31.347735021+00:00", children:[UNAVAILABLE:WSA Error {created_time:"2024-08-23T15:35:31.347692326+00:00", wsa_error:10048, grpc_status:14, os_error:"Only one usage of each socket address (protocol/network address/port) is normally permitted.\r\n", syscall:"bind"}]}]}


(RolloutWorker pid=36308) Mono path[0] = 'C:/my/code/github reposetory/Continios_Level_Design/unity_built_window_server/2/RLUnity_Data/Managed'
(RolloutWorker pid=36308) Mono config path = 'C:/my/code/github reposetory/Continios_Level_Design/unity_built_window_server/2/MonoBleedingEdge/etc'


2024-08-23 19:05:39,670	INFO trainable.py:164 -- Trainable.setup took 227.354 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2024-08-23 19:05:39,673	WARNING util.py:62 -- Install gputil for GPU system monitoring.


In [7]:
initial_state = algo.get_policy("policy_initializer").get_initial_state()
terminateds ={} 
truncateds={}
list_of_infos= []

for i in range(200): 
    obs,info=env.reset()
    terminateds['__all__'] = truncateds['__all__'] = False
    episode_reward=0
    epsiode_lenth=0
   
   
    while not  terminateds['__all__'] and not  truncateds['__all__'] :
        action= algo.compute_single_action(observation=obs['instantiator Behaviour?team=0_0'] , state=initial_state , policy_id= "policy_initializer")
        action_dict={"instantiator Behaviour?team=0_0":action}
        obs, rewards, terminateds, truncateds, infos=env.step(action_dict)
        epsiode_lenth = epsiode_lenth+1
        #print(rewards['instantiator Behaviour?team=0_0'])
  
   

    if  terminateds['__all__'] :
        list_of_infos.append(infos)
        print(infos)
        print(epsiode_lenth)
        print('\n')
          
    if  truncateds['__all__']:
        None
        


        
   
        
          
        

{'satisfied': 5.0, 'w_l': 0.36666667, 'max_percentage': 2.0}
65


{'satisfied': 5.0, 'w_l': 0.115384616, 'max_percentage': 6.0}
65


{'satisfied': 5.0, 'w_l': 0.1923077, 'max_percentage': 2.0}
65


{'satisfied': 4.0, 'w_l': -0.0625, 'max_percentage': 0.0}
65


{'satisfied': 4.0, 'w_l': 0.1, 'max_percentage': 8.0}
65


{'satisfied': 3.0, 'w_l': -0.071428575, 'max_percentage': 2.0}
65


{'satisfied': 5.0, 'w_l': -0.115384616, 'max_percentage': 8.0}
65


{'satisfied': 5.0, 'w_l': 0.115384616, 'max_percentage': 2.0}
65


{'satisfied': 5.0, 'w_l': -0.13636364, 'max_percentage': 6.0}
65


{'satisfied': 5.0, 'w_l': 0.125, 'max_percentage': 6.0}
65


{'satisfied': 5.0, 'w_l': 0.071428575, 'max_percentage': 4.0}
65


{'satisfied': 5.0, 'w_l': 0.0, 'max_percentage': 8.0}
65


{'satisfied': 5.0, 'w_l': -0.033333335, 'max_percentage': 8.0}
65


{'satisfied': 5.0, 'w_l': -0.083333336, 'max_percentage': 2.0}
65


{'satisfied': 4.0, 'w_l': -0.21428572, 'max_percentage': 0.0}
65


{'satisfied': 5.0, '

## test the policy

In [ ]:
def load_policy(filename_policy):
    mypolicy=Policy.from_checkpoint(filename_policy)
    return mypolicy




In [ ]:
mypolicy=load_policy(filename_policy)

initial_state = mypolicy.get_initial_state()
mymodel=mypolicy.model
terminateds ={} 
truncateds={}
num_win=0
num_loss=0


for i in range(30): 
    obs,info=env.reset()
    obs=obs['instantiator Behaviour?team=0_0']
    obs=obs.reshape(1,24)
    obs=torch.from_numpy(obs).to("cuda")
    input_dict={"obs":obs,"obs_flat":obs}
    seq_lens =[0]
    terminateds['__all__'] = truncateds['__all__'] = False
    episode_reward=0
    timestep=0
   
   
    while not  terminateds['__all__'] and not  truncateds['__all__'] :
        action= mymodel.forward(input_dict=input_dict , state=initial_state ,seq_lens=seq_lens )
        print(action)
        action={"element_1":action}
        action_dict={"instantiator Behaviour?team=0_0":action}
        obs, rewards, terminateds, truncateds, infos=env.step(action_dict)
        obs=obs['instantiator Behaviour?team=0_0']
        obs=obs.reshape(1,24)
        obs=torch.from_numpy(obs).to("cuda")
        input_dict={"obs":obs,"obs_flat":obs}
        #print(rewards)
        episode_reward += rewards["instantiator Behaviour?team=0_0"]
        timestep +=1
    print("total time step is {}".format(timestep))
    print("total episode_reward is {}".format(episode_reward))
    if  terminateds['__all__'] :
        print("reach the goal "+"\n")
        num_win=num_win+1
          
    if  truncateds['__all__']:
        print("not reach"+"\n")
        num_loss=num_loss+1

print("number of win is {}".format(num_win))
print("number of loss is {}".format(num_loss))

        
   
        
          
        


# test

In [8]:

dicts=list_of_infos
keys = ["difficulty","satisfied","round_count","not_playable" , "both" ]
max_percentage=5
list_of_dicts = [{keys[0]: [0]*5   , keys[1]: [0]*6 ,keys[2]: 0, keys[3]: 0,keys[4]: 0} for _ in range(max_percentage)]


#print(list_of_dicts)




for d in dicts:
        max= int(d['max_percentage'])
        if max ==0:
            max_percentage =0
        if max ==2:
            max_percentage =1
        if max ==4:
            max_percentage =2
        if max ==6:
            max_percentage =3
        if max ==8:
            max_percentage =4

    

        list_of_dicts[max_percentage]['round_count']+=1

    
        difficulty=(d["w_l"])
        if difficulty >=-0.1  and difficulty <=0.1:
            list_of_dicts[max_percentage]['difficulty'][2]+=1
      
        if difficulty >=-0.3  and difficulty <-0.1:
            list_of_dicts[max_percentage]['difficulty'][1]+=1
 
        if difficulty >=-0.5   and difficulty <-0.3:
            list_of_dicts[max_percentage]['difficulty'][0]+=1
       
        if difficulty >0.1  and difficulty <=0.3:
            list_of_dicts[max_percentage]['difficulty'][3]+=1   
         
        if difficulty >0.3  and difficulty <=0.5:
            list_of_dicts[max_percentage]['difficulty'][4]+=1  
        
        if difficulty ==-1000:
            list_of_dicts[max_percentage]["not_playable"]+=1
                
            
        if(  d['satisfied']==0):
                list_of_dicts[max_percentage]['satisfied'][0]+=1
        if(  d['satisfied']==1):
                list_of_dicts[max_percentage]['satisfied'][1]+=1
        if(  d['satisfied']==2):
                list_of_dicts[max_percentage]['satisfied'][2]+=1
        if(  d['satisfied']==3):
                list_of_dicts[max_percentage]['satisfied'][3]+=1
        if(  d['satisfied']==4):
                list_of_dicts[max_percentage]['satisfied'][4]+=1
        if(  d['satisfied']==5):
                list_of_dicts[max_percentage]['satisfied'][5]+=1     
    


        if difficulty >=-0.1  and difficulty <=0.1 and d['satisfied']==5:
                list_of_dicts[max_percentage]['both']+=1




for d in list_of_dicts:
    print(d)
    

{'difficulty': [2, 5, 22, 17, 1], 'satisfied': [0, 0, 0, 5, 6, 36], 'round_count': 47, 'not_playable': 0, 'both': 17}
{'difficulty': [0, 7, 26, 12, 2], 'satisfied': [0, 0, 0, 2, 10, 35], 'round_count': 49, 'not_playable': 2, 'both': 21}
{'difficulty': [1, 3, 18, 9, 1], 'satisfied': [0, 0, 1, 0, 5, 26], 'round_count': 34, 'not_playable': 2, 'both': 14}
{'difficulty': [2, 1, 7, 19, 5], 'satisfied': [0, 0, 0, 1, 4, 29], 'round_count': 34, 'not_playable': 0, 'both': 6}
{'difficulty': [0, 5, 13, 12, 5], 'satisfied': [0, 0, 0, 1, 12, 22], 'round_count': 36, 'not_playable': 1, 'both': 7}


In [9]:

keys = ["difficulty","satisfied","round_count","not_playable" ,"both"]
list_of_all = [{keys[0]: [0]*5  , keys[1]: [0]*6 ,keys[2]: 0, keys[3]: 0,keys[4]: 0} for _ in range(1)]



for i in range(5):
    first_elements = [d['difficulty'][i] for d in list_of_dicts]
    total_sum = sum(first_elements)  
    list_of_all[0]['difficulty'][i]=total_sum
for i in range(6):    
    satisfied = [d['satisfied'][i] for d in list_of_dicts]
    total_sum_s = sum(satisfied)  
    list_of_all[0]['satisfied'][i]=total_sum_s

all_round = [d['round_count'] for d in list_of_dicts]
total_sum_r= sum(all_round)  
list_of_all[0]['round_count']=total_sum_r

n_p = [d['not_playable'] for d in list_of_dicts]
total_sum_r= sum(n_p)  
list_of_all[0]['not_playable']=total_sum_r


both = [d['both'] for d in list_of_dicts]
total_sum_both= sum(both)  
list_of_all[0]['both']=total_sum_both

print('\n')
print(list_of_all)



[{'difficulty': [5, 21, 86, 69, 14], 'satisfied': [0, 0, 1, 9, 37, 148], 'round_count': 200, 'not_playable': 5, 'both': 65}]
